In [ ]:
!pip install skorch

In [ ]:
import os
import glob
import tqdm
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import skorch
from PIL import Image
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, roc_auc_score

## Using Pre-trained CNNs

<img src='https://i.stack.imgur.com/gI4zT.png'>

## Example: EuroSAT: A land use and land cover classification dataset

<a href='https://arxiv.org/abs/1709.00029'>Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification.</a> Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.

<img src='https://raw.githubusercontent.com/phelber/EuroSAT/master/eurosat_overview_small.jpg'>




### Download the dataset

In [ ]:
# http://madm.dfki.de/files/sentinel/EuroSAT.zip
os.system("wget http://madm.dfki.de/files/sentinel/EuroSAT.zip")
os.system("unzip EuroSAT.zip")

### Load the dataset

In [ ]:
def load_eurosat_dataset():

  data_folders = sorted(glob.glob("2750/*"))
  # preprocessing steps for image
  preprocess = transforms.Compose([transforms.ToTensor(),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.RandomVerticalFlip(),
                                   # normalization used on training resnet-50 data
                                   transforms.Normalize(mean=[0.7137, 0.6628, 0.6519], \
                                                        std=[0.2970, 0.3017, 0.2979]),])
  X = []
  y = []
  for idx, folder in enumerate(data_folders):
    imgs = sorted(glob.glob(folder + "/*.jpg"))
    for i in tqdm.tqdm(imgs):
      img = Image.open(i)
      img = preprocess(img)
      X.append(img)
      y.append(torch.tensor([idx]))
  X = torch.stack(X, dim=0).float()
  y = torch.stack(y, dim=0).flatten().long()
  return X, y

X, y = load_eurosat_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

### Initialize pre-trained model

In [ ]:
model = torchvision.models.resnet50(pretrained=True)
model

### Re-initialize final layer of pre-trained model

In [ ]:
model.fc = nn.Sequential(nn.Linear(2048, 10),
                          nn.Softmax(dim=1))

### Fit the model

In [ ]:
from skorch import NeuralNetClassifier
def optim(pgroups, **kwargs):
  return torch.optim.Adam(model.fc.parameters(), **kwargs)
clf = NeuralNetClassifier(model, batch_size=512, max_epochs=10, lr=1e-3, optimizer=optim, device='cuda')
clf.fit(X_train, y_train)

In [ ]:
clf = NeuralNetClassifier(model, batch_size=512, max_epochs=10, lr=1e-3, device='cuda')
clf.fit(X_train, y_train)

### Evaluate the model

In [ ]:
clf.score(X_test, y_test)